### Chaining and Teeing

Chaining Iterables  
  
itertool.*chain*(\*args) -> lazy iterator  
This is analagous to sequence concatentation but not the same!  
-> dealing with iterable (including iterators)  
-> chaining is itself a lazy iterator  
We can manually chain iterables this way:  
iter1 iter2 iter3  
*for it in (iter1, iter2, iter3):
    yield from it*
    
or, we can use *chain* as follows:  
*for item in chain(iter1, iter2, iter3):
    print(item)*  
    
Note the variable number of positional arguments - each argument must be an iterable!

What happens if we want to chain from iterable contained inside another, single, iterable?

l = [iter1, iter2, iter3]

chain(l) -> l

What we really want is to chain iter1, iter2, and iter3  
We can try using unpacking: chain(\*l)  
-> produced elements from iter1, iter2, and iter3  

**BUT** unpacking is eager, not lazy!

This could be a issue if we desire the entire chaining process to be lazy.

We can use itertools.chain.*from_iterable(it)* -> lazy iterator

So we \*could\* use:  
def chain_lazy(it):
    for sub_it in it:
        yield from sub_it
   
Or we can use chain.from_iterable:

chain.from_iterable(it)  

This achieves the same result  
-> iterates lazily over it  
-> in turn, iterates lazily over each iterable in it

"Copying" Iterators

itertools.*tee(iterable, n)*  
Sometimes we need to iterate through the same iterator multiple times, or even in parallel

We \*could\* create teh iterator multiple times manually

ie   
  
iters = []
for _ in range(10):
    iters.append(create_iterator())

**OR** we can use the *tee* in itertools  
-> returns independent iterators in a tuple

ie  
  
tee(iterable, 10) -> (iter1, iter2, iter3, ..., iter10)

Those iterables are all different objects!

Teeing Iterables

One important thing to note:  
The elements of the returned tuple are lazy iterators  

ie  
l = [1, 2, 3, 4]
tee(l, 3) -> (iter1, iter2, iter3)

Each of iter1, iter2, iter3 **is** a lazy iterator, despite l being a list

#### Code Examples

In [2]:
l1 = (i**2 for i in range(4))
l2 = (i**2 for i in range(4, 8))
l3 = (i**2 for i in range(8, 12))

In [3]:
for gen in l1, l2, l3:
    for item in gen:
        print(item)

0
1
4
9
16
25
36
49
64
81
100
121


In [7]:
def chain_iterables(*iterables):
    for iterable in iterables:
        yield from iterable

In [8]:
l1 = (i**2 for i in range(4))
l2 = (i**2 for i in range(4, 8))
l3 = (i**2 for i in range(8, 12))

In [9]:
for item in chain_iterables(l1, l2, l3):
    print(item)

0
1
4
9
16
25
36
49
64
81
100
121


In [10]:
from itertools import chain

In [11]:
l1 = (i**2 for i in range(4))
l2 = (i**2 for i in range(4, 8))
l3 = (i**2 for i in range(8, 12))

In [12]:
for item in chain(l1, l2, l3):
    print(item)

0
1
4
9
16
25
36
49
64
81
100
121


In [13]:
l1 = (i**2 for i in range(4))
l2 = (i**2 for i in range(4, 8))
l3 = (i**2 for i in range(8, 12))

lists = [l1, l2, l3]
for item in chain(lists):
    print(item)

<generator object <genexpr> at 0x0000020C762D2AC8>
<generator object <genexpr> at 0x0000020C762D29C8>
<generator object <genexpr> at 0x0000020C762D2BC8>


In [14]:
l1 = (i**2 for i in range(4))
l2 = (i**2 for i in range(4, 8))
l3 = (i**2 for i in range(8, 12))

lists = [l1, l2, l3]
for item in chain(lists):
    for i in item:
        print(i)

0
1
4
9
16
25
36
49
64
81
100
121


In [15]:
l1 = (i**2 for i in range(4))
l2 = (i**2 for i in range(4, 8))
l3 = (i**2 for i in range(8, 12))

lists = [l1, l2, l3]

for item in chain(*lists):
    print(item)

0
1
4
9
16
25
36
49
64
81
100
121


In [19]:
def squares():
    yield (i**2 for i in range(4))
    yield (i**2 for i in range(4, 8))
    yield (i**2 for i in range(8, 12))

In [20]:
for item in chain(*squares()):
    print(item)

0
1
4
9
16
25
36
49
64
81
100
121


In [24]:
def squares():
    print('yielding first item')
    yield (i**2 for i in range(4))
    print('yielding second item')
    yield (i**2 for i in range(4, 8))
    print('yielding third item')
    yield (i**2 for i in range(8, 12))

In [25]:
def read_values(*args):
    print('finished reading arguments')

In [26]:
read_values(squares())

finished reading arguments


In [27]:
read_values(*squares())

yielding first item
yielding second item
yielding third item
finished reading arguments


In [28]:
c = chain.from_iterable(squares())

In [29]:
for item in c:
    print(item)

yielding first item
0
1
4
9
yielding second item
16
25
36
49
yielding third item
64
81
100
121


In [30]:
def chain_iterables(*iterables):
    for iterable in iterables:
        yield from iterable

In [31]:
def chain_from_iterable(iterable):
    for item in iterable:
        yield from item

In [32]:
for item in chain_from_iterable(squares()):
    print(item)

yielding first item
0
1
4
9
yielding second item
16
25
36
49
yielding third item
64
81
100
121


In [33]:
from itertools import tee

In [34]:
def squares(n):
    for i in range(n):
        yield i**2

In [48]:
gen = squares(10)

In [49]:
gen

<generator object squares at 0x0000020C7624E148>

In [50]:
iters = tee(gen, 3)

In [51]:
iters

(<itertools._tee at 0x20c76252848>,
 <itertools._tee at 0x20c76252448>)

Notice I have 3 different objects now

In [52]:
type(iters)

tuple

In [53]:
iter1, iter2, iter3 = iters

In [54]:
iter1 is iter2

False

In [55]:
iter2 is iter3

False

In [56]:
gen is iter1

False

In [57]:
next(iter1), next(iter1), next(iter1)

(0, 1, 4)

In [58]:
next(iter2), next(iter2), next(iter2)

(0, 1, 4)

In [59]:
next(iter3), next(iter3), next(iter3)

(0, 1, 4)

In [60]:
next(gen), next(gen), next(gen)

(9, 16, 25)

Notice that gen **was** iterated through when tee was called!

In [63]:
l = [1, 2, 3, 4]
lists = tee(l, 2)

In [64]:
lists[0]

Notice that we have an iterator not iterable!

In [66]:
list(lists[0])

[1, 2, 3, 4]

In [68]:
list(lists[1])

[1, 2, 3, 4]

In [69]:
list(lists[0])

[]

In [70]:
lists[0] is lists[0].__iter__()

True

In [71]:
'__next__' in dir(lists[0])

True